## Part 1 Bulk reaction and phase diagram

Supervisor: Prof. Aschauer

Assistants: Dr. Xing Wang

Room:	N216

### 1. Introduction

In this part, we will look at the bulk system. In next part we will look at the surface system. The reaction for bulk La$_2$Ti$_2$O$_7$ and gas NH$_3$ can be written as: 

La$_2$Ti$_2$O$_7$ + 2 NH$_3$ -> LaTiO$_2$N + 3 H$_2$O

Therefore, we need to calculate the properties (energyies) of the following phases:
* Gas phases: NH$_3$, H$_2$O
* Crystalline (bulk) solid phases: La$_2$Ti$_2$O$_7$, LaTiO$_2$N



### 2. Gas phase

You will first calculate the pure molecules: NH$_3$ and H$_2$O. This is the same as calculation for O$_2$, which you have learned in the PC-II practical. Here is the python code for NH$_3$, please build your code for H$_2$O.

#### 2.1 NH$_3$

First build the atomic structure.



In [5]:
from ase.build import molecule
from x3dase.visualize import view_x3d_n

nh3 = molecule('NH3')
nh3.cell = [10, 10, 10]       # Unit cell
nh3.pbc = [True, True, True]  # We always use the periodic boundary condition for our DFT calculation
view_x3d_n(NH3, bond=1.0, label = True, output = 'htmls/nh3.html')

Then build the Espresso calculator, and carry out a 'relax' calculation.

In [ ]:
from xespresso import Espresso
from pseudo import mypseudo

# For this small molecule, we use the "debug" partition to avoid long waiting time
queue = {'ntasks': 20, 'partition': 'debug', 'time': '00:10:00'}

calc = Espresso(label = 'molecule/nh3',
                pseudopotentials = mypseudo,
                calculation = 'relax',   # allow atoms to move
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                kpts = (1, 1, 1),
                queue = queue)
nh3.calc = calc
e = nh3.get_potential_energy()
print('Energy {0:3.3f}'.format(, e))

#### 2.2 H$_2$O
Please build your code for H$_2$O

### 3. Bulk solid phases
La$_2$Ti$_2$O$_7$, LaTiO$_2$N.

#### 3.1 La$_2$Ti$_2$O$_7$
We first read the crystal structure.


In [1]:
from ase.io import read
from x3dase.visualize import view_x3d_n

lto = read('datas/La2Ti2O7.cif')
view_x3d_n(lto, bond=1.0, label = True, output = 'htmls/la2ti2o7.html')

In [2]:
 # print the unit cell 
 a, b, c, alpha, beta, gamma = lto.get_cell_lengths_and_angles()
print('Unit cell: {0:1.3f}, {1:1.3f}, {2:1.3f}'.format(a, b, c))
print('k-point:   {0:1.1f}, {1:1.1f}, {2:1.1f}'.format(25/a, 25/b, 25/c))

Unit cell: 7.817, 5.598, 13.219
k-point:   3.2, 4.5, 1.9


Then build the Espresso calculator. 

We use nk1×a, nk2×b, nk3×c ~= 25 to set the `k-point`. In the case of La$_2$Ti$_2$O$_7$, a = 7.817, b = 5.598, c = 13.210, thus we set `kpts=(3, 5, 2)`.

We need to add a new parameter (`Hubbard_U`) for `Ti` element. DFT+U method is to treat the strong on-site Coulomb interaction of localized electrons ($d$ and $f$ orbitals), which is not correctly described by the GGA functional.

In [4]:
from xespresso import Espresso
from pseudo import mypseudo

# For this big bulk system, we use the "empi" partition
queue = {'nodes': 4, 'ntasks-per-node': 20, 'partition': 'empi', 'mem-per-cpu': '5G', 'time': '23:59:00'}

input_ntyp = {
    'Hubbard_U': {'Ti': 3.0},                # DFT + U parameter
    'starting_magnetization': {'Ti': 0.0}}   # magnetization

calc = Espresso(label = 'bulk/lto',
                pseudopotentials = mypseudo,
                calculation = 'relax',   # allow atoms to move
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                kpts = (3, 5, 2),        # k-points, 
                electron_maxstep = 200,
                nspin = 2,
                lda_plus_u = True,
                input_data = {'input_ntyp': input_ntyp},   # DFT+U and magnetization
                queue = queue,
                parallel = '-nk 5',)
lto.calc = calc
lto.get_potential_energy()
e = lto.calc.results['energy']
print('Energy {0:1.3f}'.format(e))

CalculationFailed: Calculator "espresso" failed with command "sbatch .job_file" failed in /home/xing/espresso/bsc-2021-unibe/bulk/lto with error code 127

What is the optimized lattice parameters? $a, b, c, \alpha, \gamma, \beta$

In [ ]:
# read the optmized structure
lto_opt = lto.calc.results['atoms']
a, b, c, alpha, beta, gamma = lto_opt.get_cell_lengths_and_angles()
print(a, b, c, alpha, beta, gamma)

#### 3.2 LaTiO$_2$N
Please write your code for LaTiO$_2$N. The inpur file for atomic structure is in the datas folder.

### 4 Calculation the reaction energy
The reaction energy is given by

$\Delta E = E_{products} - E_{reactant}$

A negative value means that there is an energy gain for reaction, while a positive value means that reaction is not energetically favorable.


In [ ]:
#=====================================================
#  bulk reaction
# use the energy from previous optmization calculation for the structures
E_nh3 = nh3.calc.results['energy']
E_h2o = h2o.calc.results['energy']
E_lto = lto.calc.results['energy']
E_lton = lton.calc.results['energy']
dE = E_h2o + E_lton - E_lto + E_nh3
print('reaction energy (eV):  {0:1.2f}'.format(dE))

### 5. Ab initio atomistic thermodynamics
The Gibbs free energy of molecule system depends heavily on external temperature and pressure. In order to compare DFT results with experimental data, ab initio atomistic thermodynamics (AIAT) will be used.

ASE contains a `IdealGasThermo` module that allow us calculate the thermodynamic quantities of molecules: https://wiki.fysik.dtu.dk/ase/ase/thermochemistry/thermochemistry.html?highlight=thermodynamic. 

We need the vibrational energies, which can be calculated with the `vibrations` module in ASE. We first read the optmized structure for NH$_3$ and H$_2$O , and then use a new Espresso calculator with `calculation = 'scf'` instead of `relax`.


In [1]:
from ase.build import molecule
from ase.calculators.emt import EMT
from ase.optimize import QuasiNewton
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

# read the optmized structure and energy for NH3
nh3_opt = nh3.calc.results['atoms']

# For this small molecule, we use the "debug" partition to avoid long waiting time
queue = {'ntasks': 20, 'partition': 'debug', 'time': '00:10:00'}

calc = Espresso(label = 'molecule/nh3-vib',
                pseudopotentials = mypsudo,
                calculation = 'scf',   # Do not allow atoms to move
                tstress = True,
                tprnfor = True,
                ecutwfc = 40,
                occupations = 'smearing',
                degauss = 0.01,
                kpts = (1, 1, 1),
                queue = queue)
nh3_opt.calc = calc

vib_ = Vibrations(nh3_opt)
vib_nh3.run()
vib_energies_nh3 = vib_nh3.get_energies()


NameError: name 'nh3' is not defined

Please write the code for H$_2$O.

Now we calculate the Gibbs free energy for NH$_3$ and H$_2$O molecules at given a temperature (K) and pressure (Pa). Please do a literature review to find the reaction coditions (T and P) in the experiment. 

In [ ]:
# Temperature
T = 1000

# NH3
P_nh3 = 1e5  # unit Pa
thermo_nh3 = IdealGasThermo(vib_energies=vib_energies_nh3,    # use the vib_energies_nh3 from previous vibrational calculation
                        potentialenergy=E_nh3,                # use the optimized energy from previous optmization calculation
                        atoms=nh3,
                        geometry='nonlinear',
                        symmetrynumber=3, spin=0)
G_nh3 = thermo_nh3.get_gibbs_energy(temperature=T, pressure=P_nh3)
# H2O
P_h2o = 1e5  # unit Pa
thermo_h2o = IdealGasThermo(vib_energies=vib_energies,
                        potentialenergy=potentialenergy,
                        atoms=h2o,
                        geometry='nonlinear',
                        symmetrynumber=2, spin=0)
G_h2o = thermo_nh3.get_gibbs_energy(temperature=T, pressure=P_h2o)

Calculation the reaction energy at the given temperature (K) and pressure (Pa).

In [ ]:
# For the gas phase, we use Gibbs free energy G_nh3 and G_h2o instead of E_nh3 and E_h2o
dG = G_h2o + E_lton - E_lto + G_nh3
print('Temperature: {0}'.format(T))
print('Pressure NH3: {1}'.format(P_nh3))
print('Pressure H2O: {1}'.format(P_h2o))
print('Reaction Gibbs free energy (eV):  {0:1.2f}'.format(site, dG))

### 6. Phase diagram
A phase diagram shows the stable phases for a given system in a wide range of conditions. Phase diagram is an important tool in materials science, physical chemistry and mineralogy, and can be thought of as a "map" for researchers.

In order to construct a phase diagram, the boundary between the La$_2$Ti$_2$O$_7$ and LaTiO$_2$N should be found. we need solve the following equation:
\begin{equation}
\Delta G(T, P) = 0,
\end{equation}
The equilibrium point ($T$, $P$) is then plotted to construct the phase boundary.

In [ ]:
import numpy as np
P_h2o = 1e5  # unit Pa

# Temperature range
T = np.linspace(300, 1500, 10)

P = []
for t in T:
    # def the function to solve the equation
    def calcdG(T):
        G_nh3 = thermo_nh3.get_gibbs_energy(temperature=T, pressure=P_nh3)
        G_h2o = thermo_nh3.get_gibbs_energy(temperature=T, pressure=P_h2o)
        dG = G_h2o + E_lton - E_lto + G_nh3
        return dG
